In [43]:
# !pip install PyPDF2
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org wand -vvv
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org config -vvv
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org MagickWand
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org ImageMagick
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pip
# !pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org google-cloud-vision -vvv
# !pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pytesseract -vvv
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pandas --upgrade
#import tensorflow as tf
#import tflite_runtime.interpreter as tflite

In [36]:
import os
import re
import config
import pandas as pd
from datetime import datetime
import pytesseract
import traceback
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\debi.rath\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
from PIL import Image

config.invoice_list = []
config.file_name = 'NA'


def process_invoice_image(img):
    try:
        inv_data = pytesseract.image_to_string(Image.open(img), lang = "eng")
    except Exception as e:
        pass
    try:
        with open('invoice_scan.txt','w'):
            logger("====>clean file.")
            file = open('invoice_scan.txt','a')
            file.write(inv_data)
            file.write('\n')
            file.close()
    except Exception as e:
        track = traceback.format_exc()
        logger('====>method: process_invoice_image(),Exception :'+ "\n" + str(track))

def handle_invoice_image(img):
    try:
        data = process_invoice_image(img)
        with open('invoice_scan.txt','r') as file:
            test_str = file.read()
            if re.search('tax\s+invoice',test_str,re.I):
                print("=====>loading invoice data"+ "\n" + test_str)
                parse_invoice_image(test_str)
    except Exception as e:
            track = traceback.format_exc()
            logger('=====>method: handle_invoice_image(), Exception :' + "\n" +str(track))




def parse_invoice_image(data):
    filename = config.file_name
    invoice_dict = {}
    test_str = str(data) 
    customer_name,invoice_no, date, total_amount = ([''],) * 4
    try:
        customer_name = re.findall(r"(?:MR\.|MRS\.|MIS\.|M\/S/\.|SRI\.|MISS\.|MS\.|WS\.)[\r\n\s](.*?)(?=Date/Time)",test_str, re.I | re.S)
        customer_name = [re.sub(r"Mr.|Mrs.|Miss.|MISS.|M/S.|WS|MR.|MRS.|SRI.|Sri.|\n|\.", "", customer_name[0]).strip()]
    except Exception as e:
        track = traceback.format_exc()
        print("====>ERROR WHILE FETCHING" + "\n" + str(track))
    try:
        invoice_no = re.findall(r"Invoice\s+No[\r\n\s.:]+([^\r\n]+)", test_str, re.I)
    except Exception as e:
        track = traceback.format_exc()
        print("====>ERROR WHILE FETCHING" + "\n" + str(track))
    try:
        date = [re.findall(r"Date.*Time[\r\n :]+([^\r\n]+)", test_str, re.I)[0].split(" ")[0]]
    except Exception as e:
        track = traceback.format_exc()
        print("====>ERROR WHILE FETCHING" + "\n" + str(track))
    try:
        total_amount = re.findall(r"Invoice\s+Amount[\r\n\s.:Rs.]+([^\r\n]+)", test_str, re.I)
    except Exception as e:
        track = traceback.format_exc()
        print("====>ERROR WHILE FETCHING" + "\n" + str(track))
    processed_date = datetime.now().strftime('%d/%m/%Y')
    file_name = re.findall(r'(?<=)(.*)(?=.PNG)',filename)[0]
    invoice_dict['Customer_Name'] = customer_name[0] if customer_name else ''
    invoice_dict['Invoice_No'] = invoice_no[0] if invoice_no else ''
    invoice_dict['Date'] = date[0] if date else ''
    invoice_dict['Total_Amount'] = total_amount[0] if total_amount else ''
    invoice_dict['File_Name'] = file_name
    invoice_dict['Processed_Date'] = processed_date
    invoice_dict['Invoice_Type'] = "Final"
    invoice_dict = {k: v if v else 'NA' for k, v in invoice_dict.items()}
    config.invoice_list.append(invoice_dict)
    
    

def logger(data):
    print(data)

# def output_file(data):
#     invoice_list = parse_invoice_image(data)
#     df = pd.DataFrame(invoice_list)
#     df.to_csv(r'master_invoice_automated.csv', index=False)

def invoice_scanner():
    base_dir = r'C:\\Users\\debi.rath\\Downloads\\InvoiceExtraction'
    if not os.path.exists(base_dir+f'''/resource'''):
        os.makedirs(base_dir+f'''/resource''')
    path = os.path.join(base_dir, f'''resource''')
    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.PNG'):
                filename = os.path.join(path,file.name)
                logger("====>File Name:"+ "\n"+filename)
                if(filename.find("Invoice") != -1):
                    config.file_name = file.name
                    handle_invoice_image(filename)
        logger("====>invoice extraction process completed successfully.")
        logger(config.invoice_list)
        df = pd.DataFrame(config.invoice_list)
        logger("====>extracted invoice features are as below.")
        logger(df)
        df.to_csv(r'master_invoice_automated.csv', index=False)
        config.invoice_list = []
        logger("====>clean up invoice_list again to proceed further.")
        logger(config.invoice_list)
if __name__ == '__main__':
    invoice_scanner()

====>File Name:
C:\\Users\\debi.rath\\Downloads\\InvoiceExtraction\resource\Anant-Invoice.PNG
====>clean file.
=====>loading invoice data
TAX INVOICE

Casomer 20012615 ini No 100000147
Mr ARVINO RAD PATIL pia rioaaa20 1348
S50 PUNDLICRAO PATIL Scr ett Sra

2.68 SUNSHINE SLOK COLONY path ee
AB ROAD PIGADAMBAR Order Date. 06/02/2020
BORE Dealer GSTIN aoaaleseaset12U
MADHYA PRADESH customer GSTINUIN = WA

Mobile No, -9893150203,

Hire Purchase/Leased To/Hypothecated To/Lessor: TOYOTA FINANCIAL SERVICES INDIA LTD, GWALIOR A
PARTICULARS AMOUNT(RS.)

Debit Credit

INNOVA CRYSTA24G 17,63513.51
PACKAGE: X. 8 SEATER

 

     

Price of one

HSN Number: 8703

VIN No. : MBJJB8EM801578441~0120
Engine No. -2GD A393072
KeyNo. 45185
ExtColor AVANT GARDE BRONZE
IntColor INT CLR FOR INNOVA CRYSTA - BL
Discount 0.00
Unit Price after discount 1163513.51
Taxes:
“1 CGST @14.00% 1,62,891.89,
2 UTGST/SGST @14.00% 162,891.89

3. Compensation Cess @20.00%


====>ERROR WHILE FETCHING
Traceback (most recent cal

====>clean file.
=====>loading invoice data
Customer :120043653

Mr. K SATHISH RAI

S/O MR K GUDDAPPA RAI

D.NO: 4-58, KUTHAMOTTE HOUSE
UBBARADKA MITHUR

PUTTUR- 574248

KARNATAKA

Mobile No, :8197818211
Hire Purchase/L eased To/Hypothecated To/Lessor: TOYOTA FINANCIAL SERVICES INDIA LTD, BANGALORE

TAX INVOICE
Invoice No.
Date/Time
Delivery Dealer
Order No.
Order Date

Dealer GSTIN
Customer GSTIN/UIN

 

PARTICULARS

 

 

1Nv200000049
29/08/2020 18:21
UUOTA
0R0200000079
29/06/2020
29AAACU4250F 126

NA

AMOUNT(RS.)

a Debit Credit
Price of ene :TOYOTA GLANZA G (MT) = 5,48,837.21
u PACKAGE: 5 SEATER
HSN Number: 8703
VINNo.  :MBHGWB22SLC476855
Engine No. :K12MP 4033934
KeyNo. 50925
Ext.Color :CAFE WHITE
Int.Color :2 TONE BLACK/BLUE
Discount 7781.94
Unit Price after discount : 54108527
Texes:
1 CGST @14.00% 78,751.94
2 UTGSTISGST @14.00% 75,751.94
3 Compensation Cess @1.00% 5.41085

 

‘TOTAL(DebitCredit)(Rs.)

7,05,751.94 __7,751.94

 

Invoice Amount :

 

Rs.6,98,000.00


====>File N